In [1]:
!git clone https://github.com/kpu/kenlm.git

Cloning into 'kenlm'...
remote: Enumerating objects: 14185, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 14185 (delta 117), reused 80 (delta 80), pack-reused 14010 (from 2)
Receiving objects: 100% (14185/14185), 5.91 MiB | 19.92 MiB/s, done.
Resolving deltas: 100% (8059/8059), done.


In [2]:
%cd kenlm
!mkdir -p build
%cd build
!cmake ..
!make -j4

/content/kenlm
/content/kenlm/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.74.0/BoostConfig.cmake (found suitable version "1.74.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Found Threads: TRUE  
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Found BZip2: /usr/lib/x86_64-linux-gnu/libbz2.so (found version "1.0.8") 
-- Looking for BZ2_bzCo

In [4]:
!/content/kenlm/build/bin/lmplz -o 3 < /content/cleaned_for_kenlm.txt > /content/3gram.arpa
!/content/kenlm/build/bin/build_binary /content/3gram.arpa /content/3gram.binary

=== 1/5 Counting and sorting n-grams ===
Reading /content/cleaned_for_kenlm.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 963363 types 114582
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1374984 2:99959750656 3:187424538624
Statistics:
1 114582 D1=0.785554 D2=1.05539 D3+=1.3816
2 313477 D1=0.833607 D2=0.937082 D3+=2.24476
3 375398 D1=2.20537e-06 D2=2 D3+=3
Memory estimate for binary LM:
type       kB
probing 16855 assuming -p 1.5
probing 19139 assuming -r models -p 1.5
trie     8673 without quantization
trie     5824 assuming -q 8 -b 8 quantization 
trie     8221 assuming -a 22 array pointer compression
trie     5372 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:1374984 2:5015632 3:7507960
----5---1

In [5]:
!pip install kenlm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.3.0-cp311-cp311-linux_x86_64.whl size=3185096 sha256=3345b5635b06c35d7a9172ebbf01342a50509e6c857be4ab74d282928d8be154
  Stored in directory: /root/.cache/pip/wheels/eb/fd/7c/6630a05c6fe34e23f266b8156e4c585a5ae487f8fd7c04a3ef
Successfully built kenlm


In [8]:
import kenlm

model = kenlm.LanguageModel('3gram.binary')

sentence = "ιωακαβανίζω"
score = model.score(sentence)
print(f"Score: {score}")

Score: -6.601880073547363


In [11]:
# !pip install datasets==3.6.0

In [13]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Elormiden/Thesaurus-Cypriot-Greek-Dialect")

In [14]:
ds

DatasetDict({
    train: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 21971
    })
    validation: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 2746
    })
    test: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 2747
    })
})

In [16]:
import re
from datasets import load_dataset

def clean_text_for_dataset(text):
    """
    Очищает строку текста, удаляя этимологические маркеры, специальные символы,
    числа и слова с числами, а также лишние пробелы.
    """
    if not isinstance(text, str):
        return text

    # Шаг 1: Удаление этимологических маркеров и сокращений
    regex_to_remove = re.compile(
        r'\b(α|β|γ|δ|ε|η|θ|ι|κ|λ|μ|ν|ξ|ο|π|ρ|σ|τ|υ|φ|χ|ψ|ω|αόρ|αμ|ανθ|επίθ|επίρρ|μτφ|συνών|αντίθ|φρ|κν|αραβ|γαλλ|ενετ|ιταλ|λατ|τουρκ|θρκ|κερ|μκερ|μαπολλ|μτγν|πρβ|αί|αΐιΐηι|ΐ|ΐί|τ|ΐίηιιηι|ϊίΐι|ίϊ|ως|ώσπερ)\b',
        re.IGNORECASE | re.UNICODE
    )
    cleaned_text = regex_to_remove.sub(' ', text)

    # Шаг 2: Удаление всех чисел и слов, содержащих числа
    cleaned_text = re.sub(r'\S*\d\S*', ' ', cleaned_text)

    # Шаг 3: Замена специальных символов и пунктуации на пробелы
    cleaned_text = re.sub(r'[^\s\u0386-\u03ce]', ' ', cleaned_text, flags=re.UNICODE)

    # Шаг 4: Удаление лишних пробелов и очистка строки
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

# Имя датасета на Hugging Face
dataset_name = 'Elormiden/Thesaurus-Cypriot-Greek-Dialect'

# Загружаем датасет
dataset = load_dataset(dataset_name)

# --- Показываем пример ДО очистки ---
sample_before = dataset['train'][0]
print("--- Пример ДО очистки ---")
print(f"Original description: '{sample_before['description']}'")
print(f"Original greek_description: '{sample_before['greek_description']}'")
print("-" * 25)

# --- Применяем функцию очистки к нужным колонкам ---
cleaned_dataset = dataset.map(
    lambda example: {
        'description': clean_text_for_dataset(example['description']),
        'greek_description': clean_text_for_dataset(example['greek_description'])
    }
)

# --- Показываем пример ПОСЛЕ очистки ---
sample_after = cleaned_dataset['train'][0]
print("--- Пример ПОСЛЕ очистки ---")
print(f"Cleaned description: '{sample_after['description']}'")
print(f"Cleaned greek_description: '{sample_after['greek_description']}'")
print("-" * 25)

print("Датасет успешно очищен и готов к использованию!")

--- Пример ДО очистки ---
Original description: 'σκόρδα, η (Μ424.8,660.14) [ιταλ. εηυαάτε] μοίρα, ίλη, προσωπικό πληρώματος,'
Original greek_description: 'Σκορδα,η Με κλ.π.'
-------------------------


Map:   0%|          | 0/21971 [00:00<?, ? examples/s]

Map:   0%|          | 0/2746 [00:00<?, ? examples/s]

Map:   0%|          | 0/2747 [00:00<?, ? examples/s]

--- Пример ПОСЛЕ очистки ---
Cleaned description: 'σκόρδα εηυαάτε μοίρα ίλη προσωπικό πληρώματος'
Cleaned greek_description: 'Σκορδα Με κλ'
-------------------------
Датасет успешно очищен и готов к использованию!


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 21971
    })
    validation: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 2746
    })
    test: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 2747
    })
})

In [18]:
train_ds = dataset['train']

In [24]:
train_ds['word'][1]

'ξυλόκοτα'

In [20]:
clean_train_ds = cleaned_dataset['train']

In [25]:
clean_train_ds['word'][1]

'ξυλόκοτα'

In [26]:
cleaned_dataset

DatasetDict({
    train: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 21971
    })
    validation: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 2746
    })
    test: Dataset({
        features: ['word', 'description', 'greek_word', 'greek_description'],
        num_rows: 2747
    })
})

In [28]:
cleaned_dataset.push_to_hub("Elormiden/Thesaurus-Cypriot-Greek-Dialect")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  48%|####8     | 2.10MB / 4.37MB            

...

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  98%|#########8|  525kB /  534kB            

CommitInfo(commit_url='https://huggingface.co/datasets/Elormiden/Thesaurus-Cypriot-Greek-Dialect/commit/58f5b11563ec915b330438020304b56522d4c0fd', commit_message='Upload dataset', commit_description='', oid='58f5b11563ec915b330438020304b56522d4c0fd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Elormiden/Thesaurus-Cypriot-Greek-Dialect', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Elormiden/Thesaurus-Cypriot-Greek-Dialect'), pr_revision=None, pr_num=None)